<a href="https://colab.research.google.com/github/Chunshan-Theta/dataflowr/blob/master/Notebooks/01_intro_DLDIY_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 用 CNN 來辨識狗狗和貓貓

- 我們將創建一個模型來參加Kaggle的[Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)競賽。

這場比賽提供25,000張被標示為狗或貓照片來訓練模型，我們將使用該模型去為比賽中提供的12,500張照片組成的測試集標示標籤。

根據Kaggle網站的資料，當比賽開始時（2013年底）：
```
技術水平：根據目前的文獻表明，機器分類器可以在此任務上獲得80％以上的準確性
```

因此，如果您的模型可以超過80％的準確性，那麼您就是2013年巔峰！

##  導入


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import time
%matplotlib inline
torch.__version__

In [ ]:
import sys
sys.version



確保使用GPU進行訓練，假若沒有請遵照此[設置](https://jovianlin.io/pytorch-with-gpu-in-google-colab/).

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Using gpu: %s ' % torch.cuda.is_available())

## 下載資料集



你可以直接下載來自Kaggle的完整資料集
或是使用`Jeremy Howard`提供的[catvsdogs連結](http://files.fast.ai/data/dogscats.zip)

他區分了貓和狗的照片並放進各自的資料夾，同時也建立了驗證資料夾。

你將會需要使用這樣的資料夾架構來執行VGG模型

為了能滿足測試目的（或是你必須使用cpu去跑），你會需要使用小資料集。





In [ ]:
%mkdir data
%cd /content/data/
!wget http://files.fast.ai/data/dogscats.zip

In [ ]:
!unzip dogscats.zip

In [ ]:
%ls

In [ ]:
%cd dogscats/
%ls

## 資料處理

In [ ]:
%cd ..

In [ ]:
data_dir = '/content/data/dogscats'


`datasets`是`torchvision`函式庫的一個用來處理仔入資料用的類別（詳細請看[torchvision.datasets](http://pytorch.org/docs/master/torchvision/datasets.html)）

它整合圖片多工讀取器，可以讀取本地圖像並將其組合成一個個的小包裝，在每一次的`_forward_`和`_backward_`程序結束之後，不斷不斷提供新的小包裝給GPU進行運算

圖像需要經過一些準備，才能通過網絡。它們需要具有所有相同的大小 $224\times 224 \times 3$，以及下面通過規範化轉換完成的一些額外格式（稍後說明）。

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

vgg_format = transforms.Compose([
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

In [ ]:
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), vgg_format)
         for x in ['train', 'valid']}

In [ ]:
os.path.join(data_dir,'train')

Jupyter Notebook上的互動幫助。
> 感謝`?`指令


In [ ]:
?datasets.ImageFolder

我們看到 `datasets.ImageFolder` 有幾個屬性: `classes`, `class_to_idx`, 
`imgs`.

我們來看看它們分別是什麼

In [ ]:
dsets['train'].classes

In [ ]:
dsets['train'].class_to_idx

In [ ]:
dsets['train'].imgs[:5]

In [ ]:
dset_sizes = {x: len(dsets[x]) for x in ['train', 'valid']}
dset_sizes

In [ ]:
dset_classes = dsets['train'].classes


`torchvision`函式庫允許針對輸入資料使用複雜的前處理/轉化（例如：正規化\裁切、翻轉、選轉等等）

我們可以在`torchvision.transforms.Compose`方法中找尋到這一系列的幫助。

到[torchvision.transforms文件](http://pytorch.org/docs/master/torchvision/transforms.html)觀看細節



神奇指令 `?` 讓你可以探索你所有已經定義和忘記的函數!

In [ ]:
?vgg_format

In [ ]:
loader_train = torch.utils.data.DataLoader(dsets['train'], batch_size=64, shuffle=True, num_workers=6)

In [ ]:
?torch.utils.data.DataLoader

In [ ]:
loader_valid = torch.utils.data.DataLoader(dsets['valid'], batch_size=5, shuffle=False, num_workers=6)

In [ ]:
count = 1
for data in loader_valid:
    print(count, end=',')
    if count == 1:
        inputs_try,labels_try = data
    count +=1

In [ ]:
labels_try

In [ ]:
inputs_try.shape

顯示圖像的簡單函數：

In [ ]:
def imshow(inp, title=None):
#   Imshow for Tensor.
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = np.clip(std * inp + mean, 0,1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
?torchvision.utils.make_grid

In [ ]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs_try)

imshow(out, title=[dset_classes[x] for x in labels_try])

你電腦上的圖像是什麼呢？

In [ ]:
inputs_try

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(loader_train))

n_images = 8

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

In [ ]:
# Get a batch of validation data
inputs, classes = next(iter(loader_valid))

n_images = 8

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

## 建立 VGG 模型

`torchvision`函數庫帶有一個已預先訓練過的CNN模型，訓練是使用[ImageNet](http://www.image-net.org/)的1.2M張訓練圖像

第一次呼叫時可能會需要下載
> ```pretrained=True``` 模型將會通過網路下載 ```~/.torch/models```.

之後呼叫，模型會直接從這裡載入資料

In [ ]:
model_vgg = models.vgg16(pretrained=True)

我們將首先使用VGG模型，而無需進行任何修改。為了解釋結果，我們需要導入1000個ImageNet類別，可在以下位置找到： [https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json](https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json)

```
{"0": ["n01440764", "tench"], "1": ["n01443537", "goldfish"], "2": ["n01484850", "great_white_shark"], "3": ["n01491361", "tiger_shark"], "4": ["n01494475", "hammerhead"], "5": ["n01496331", "electric_ray"], ...
```

In [ ]:
!wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

In [ ]:
import json

fpath = '/content/data/imagenet_class_index.json'

with open(fpath) as f:
    class_dict = json.load(f)
dic_imagenet = [class_dict[str(i)][1] for i in range(len(class_dict))]

In [ ]:
dic_imagenet[:4]

In [ ]:
# five pic
inputs_try , labels_try = inputs_try.to(device), labels_try.to(device)

model_vgg = model_vgg.to(device)

In [ ]:
outputs_try = model_vgg(inputs_try)

In [ ]:
outputs_try

In [ ]:
outputs_try.shape

為了將網絡的輸出轉換為**概率**，我們通過一個 [Softmax function](https://en.wikipedia.org/wiki/Softmax_function)

In [ ]:
m_softm = nn.Softmax(dim=1)
probs = m_softm(outputs_try)
vals_try,preds_try = torch.max(probs,dim=1)

In [ ]:
torch.sum(probs,1)

In [ ]:
vals_try

In [ ]:
print([dic_imagenet[i] for i in preds_try.data])

In [ ]:
out = torchvision.utils.make_grid(inputs_try.data.cpu())

imshow(out, title=[dset_classes[x] for x in labels_try.data.cpu()])

### 調整最後一層，並且將所有層的梯度設為`false`

In [ ]:
print(model_vgg)

在本課程的後面，我們將學習這些不同的塊的功能。現在先了解以下內容：

- 卷積層(Convolution layers)用於查找圖像中的中小型圖案 -- 本地分析圖像
- 密集層(fully connected)用於組合圖像中的圖案 -- 全局分析圖像
- 池化層(Pooling layers)向下採樣 -- 為了減小圖像尺寸並改善學習特徵的不變性

![vgg16](https://mlelarge.github.io/dataflowr/Notebooks/vgg16.png)

在此實際示例中，我們的目標是使用已經訓練好的模型，僅更改輸出類的數量。為此，我們將訓練了1000個類型的最後一個“ nn.Linear”層替換為2個類型。

而為了在訓練期間凍結其他層的權重，我們設置字段`required_grad = False`。

以這種方式，在反向傳播期間將不會為它們計算梯度，因此不會更新權重。僅新替換上去2類型層的權重將被更新。

In [ ]:
for param in model_vgg.parameters():
    param.requires_grad = False
model_vgg.classifier._modules['6'] = nn.Linear(4096, 2)
model_vgg.classifier._modules['7'] = torch.nn.LogSoftmax(dim = 1)

PyTorch 的[LogSoftmax文件](https://pytorch.org/docs/stable/nn.html#logsoftmax)

In [ ]:
print(model_vgg.classifier)

In [ ]:
model_vgg = model_vgg.to(device)

## Training fully connected module

### 創建損失函數（loss function）和優化器（optimizer）


PyTorch 文件中的損失函數模組 [NLLLoss](https://pytorch.org/docs/stable/nn.html#nllloss) 和 優化器模組[torch.optim module](https://pytorch.org/docs/stable/optim.html#module-torch.optim)

In [ ]:
criterion = nn.NLLLoss()
lr = 0.001
optimizer_vgg = torch.optim.SGD(model_vgg.classifier[6].parameters(),lr = lr)

### 訓練模型

In [ ]:
def train_model(model,dataloader,size,epochs=1,optimizer=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs,classes in dataloader:
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            loss = criterion(outputs,classes)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))

In [ ]:
%%time
train_model(model_vgg,loader_train,size=dset_sizes['train'],epochs=2,optimizer=optimizer_vgg)

In [ ]:
def test_model(model,dataloader,size):
    model.eval()
    predictions = np.zeros(size)
    all_classes = np.zeros(size)
    all_proba = np.zeros((size,2))
    i = 0
    running_loss = 0.0
    running_corrects = 0
    for inputs,classes in dataloader:
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        loss = criterion(outputs,classes)           
        _,preds = torch.max(outputs.data,1)
            # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == classes.data)
        predictions[i:i+len(classes)] = preds.to('cpu').numpy()
        all_classes[i:i+len(classes)] = classes.to('cpu').numpy()
        all_proba[i:i+len(classes),:] = outputs.data.to('cpu').numpy()
        i += len(classes)
    epoch_loss = running_loss / size
    epoch_acc = running_corrects.data.item() / size
    print('Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch_loss, epoch_acc))
    return predictions, all_proba, all_classes

In [ ]:
predictions, all_proba, all_classes = test_model(model_vgg,loader_valid,size=dset_sizes['valid'])

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(loader_valid))

out = torchvision.utils.make_grid(inputs[0:n_images])

imshow(out, title=[dset_classes[x] for x in classes[0:n_images]])

In [ ]:
outputs = model_vgg(inputs[:n_images].to(device))
print(torch.exp(outputs))

In [ ]:
classes[:n_images]

## 通過預計算功能加快模型學習速度

In [ ]:
x_try = model_vgg.features(inputs_try)

In [ ]:
x_try.shape

In [ ]:
def preconvfeat(dataloader):
    conv_features = []
    labels_list = []
    for data in dataloader:
        inputs,labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        x = model_vgg.features(inputs)
        conv_features.extend(x.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
    return (conv_features,labels_list)

In [ ]:
%%time
conv_feat_train,labels_train = preconvfeat(loader_train)

In [ ]:
conv_feat_train.shape

In [ ]:
%%time
conv_feat_valid,labels_valid = preconvfeat(loader_valid)

###創建一個新的數據加載器

我們將不再加載圖像，因此我們需要構建自己的數據加載器。

In [ ]:
dtype=torch.float
datasetfeat_train = [[torch.from_numpy(f).type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(conv_feat_train,labels_train)]
datasetfeat_train = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_train]
loaderfeat_train = torch.utils.data.DataLoader(datasetfeat_train, batch_size=128, shuffle=True)

In [ ]:
%%time
train_model(model_vgg.classifier,dataloader=loaderfeat_train,size=dset_sizes['train'],epochs=50,optimizer=optimizer_vgg)

In [ ]:
datasetfeat_valid = [[torch.from_numpy(f).type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(conv_feat_valid,labels_valid)]
datasetfeat_valid = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_valid]
loaderfeat_valid = torch.utils.data.DataLoader(datasetfeat_valid, batch_size=128, shuffle=False)

In [ ]:
predictions, all_proba, all_classes = test_model(model_vgg.classifier,dataloader=loaderfeat_valid,size=dset_sizes['valid'])

## 4.查看模型預測（定性分析） 

我們要查看的最重要指標是驗證集，因為我們要檢查過度擬合。 

對於我們的第一個模型，在開始擔心如何處理之前，我們應該嘗試過度擬合 - 如果您仍然擬合不足，甚至沒有必要考慮正則化，數據擴充等！ 
（我們將在2週休息後繼續研究這些技術...） 


除了查看總體指標之外，查看以下每個示例也是一個好主意： 

    1.   隨機一些正確的標籤 
    2.   隨機一些不正確的標籤 
    3.   每個類別的最正確標籤（即，最有可能正確的標籤） 
    4.   每個類別中最不正確的標籤（即，最有可能出現錯誤的標籤） 
    5.   最不確定的標籤（即概率最接近0.5的標籤）。 

通常，這些對於調試模型中的問題特別有用。由於我們的模型非常簡單，因此在此階段可能沒有太多東西需要學習...

In [ ]:
# Number of images to view for each visualization task
n_view = 8

In [ ]:
correct = np.where(predictions==all_classes)[0]

In [ ]:
len(correct)/dset_sizes['valid']

In [ ]:
from numpy.random import random, permutation
idx = permutation(correct)[:n_view]

In [ ]:
idx

In [ ]:
loader_correct = torch.utils.data.DataLoader([dsets['valid'][x] for x in idx],batch_size = n_view,shuffle=True)

In [ ]:
for data in loader_correct:
    inputs_cor,labels_cor = data

In [ ]:
# Make a grid from batch
out = torchvision.utils.make_grid(inputs_cor)

imshow(out, title=[l.item() for l in labels_cor])

In [ ]:
from IPython.display import Image, display
for x in idx:
    display(Image(filename=dsets['valid'].imgs[x][0], retina=True))

In [ ]:
incorrect = np.where(predictions!=all_classes)[0]
for x in permutation(incorrect)[:n_view]:
    #print(dsets['valid'].imgs[x][1])
    display(Image(filename=dsets['valid'].imgs[x][0], retina=True))

In [ ]:
#3. The images we most confident were cats, and are actually cats
correct_cats = np.where((predictions==0) & (predictions==all_classes))[0]
most_correct_cats = np.argsort(all_proba[correct_cats,1])[:n_view]

In [ ]:
for x in most_correct_cats:
    display(Image(filename=dsets['valid'].imgs[correct_cats[x]][0], retina=True))

In [ ]:
#3. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((predictions==1) & (predictions==all_classes))[0]
most_correct_dogs = np.argsort(all_proba[correct_dogs,0])[:n_view]

In [ ]:
for x in most_correct_dogs:
    display(Image(filename=dsets['valid'].imgs[correct_dogs[x]][0], retina=True))

# 結論

最後我們做了什麼？簡單的邏輯回歸！如果連接不清楚，
我們將在下一課程中用一個簡單得多的示例進行說明。 
我們可能用大錘殺死了一隻蒼蠅。

![mouche](https://mlelarge.github.io/dataflowr-web/images/mouche.jpg)

在我們的案例中，大錘是在Imagenet上經過VGG預先訓練的，Imagenet是一個包含許多貓和狗圖片的數據集。確實，我們看到，無需修改，網絡就可以預測狗和貓的品種。因此，由VGG計算出的特徵對於我們的分類任務非常準確並不奇怪。最後，我們只需要學習最後一個線性層的參數，即8194個參數（不要忘記偏差$ 2 \乘以4096 + 2 $）。

確實，這可以在CPU上完成而沒有任何問題。



儘管如此，該示例仍具有啟發性，因為它顯示了深度學習項目中的所有必要步驟。
在這裡，我們並沒有為深度網絡的學習過程而苦惱，而是完成了所有初步的工程任務：


下載數據集、設置使用GPU的環境、準備數據、使用預先訓練的VGG計算功能，保存它們在您的驅動器上，以便您可以將它們用於以後的實驗。

這些步驟對於任何深度學習項目都是必不可少的，也是在使用網絡體系結構並理解學習過程之前的必要要求。

### 保存預卷積特徵

In [ ]:
!pip install -U bcolz

In [ ]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]


%mkdir /content/data/dogscats/vgg16


In [ ]:
save_array(os.path.join(data_dir,'vgg16','feat_train.bc'),conv_feat_train)
save_array(os.path.join(data_dir,'vgg16','labels_train.bc'),labels_train)
save_array(os.path.join(data_dir,'vgg16','feat_val.bc'),conv_feat_valid)
save_array(os.path.join(data_dir,'vgg16','labels_val.bc'),labels_valid)

### 上載預先計算的功能

本部分將允許您將預先計算的功能存儲在Google驅動器上，以備後用。

In [ ]:
%cd /content/data/dogscats/
!zip -r vgg16 vgg16/*

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
upload = drive.CreateFile({'title': 'vgg16_drive.zip'})
upload.SetContentFile('vgg16.zip')
upload.Upload()
print('Uploaded file with ID {}'.format(upload.get('id')))